In [ ]:
import transformers
import torch

torch.backends.cuda.enable_mem_efficient_sdp(False)
torch.backends.cuda.enable_flash_sdp(False)

model_id = "/kaggle/input/llama-3.1/transformers/8b-instruct/2"

pipeline = transformers.pipeline(
    "text-generation",
    model=model_id,
    model_kwargs={"torch_dtype": torch.bfloat16},
    device_map="auto",
)

In [ ]:
prompt_template = [
    {"role": "system", "content": "You are an AI that extracts key highlights from a recommendation letter. \
                                Given the text of a recommendation letter and a specific role (e.g., 'software engineer', 'project manager'), \
                                return the most important endorsements mentioned in the letter that are relevant to the role. \
                                Format your response in a short bullet-point list. Do not give any other information."},
    {"role": "user", "content": ""},
]

for i, row in df.iterrows():
    prompt = prompt_template.copy()
    lor_data = row["lor_data"]
    if lor_data == "":
        df["reco_condensed"] = ""
        continue
    prompt[1]["content"] = "Role :" + row["role"] + "\n Recommendations: " + lor_data + "\n Highlights: "
    outputs = pipeline(prompt, max_new_tokens = 90)
    print(outputs[0]["generated_text"][-1]["content"])
    print(lor_data)
    break
    df.loc[i, "reco_condensed"] = outputs[0]["generated_text"][-1]["content"]